In [2]:
import numpy as np
import pandas as pd
from functools import reduce

In [3]:
#bakery = pd.read_csv("5000-out2.csv", names=np.arange(50))
bakery = pd.read_csv("75000-out2.csv", names=np.arange(50))
bakery

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74996,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
74997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
74998,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
74999,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
goods = pd.read_csv("goods.csv")

In [13]:
bakery[7][2]

1

In [56]:
#sum(bakery[0])
support = np.array([sum(bakery[i]) for i in bakery.columns])
rsupport = support/len(bakery[0])

In [57]:
support

array([6265, 6271, 5626, 6157, 6948, 6168, 3178, 8193, 3189, 6206, 3153,
       5784, 5117, 3177, 6362, 5690, 6221, 5529, 6987, 5685, 3253, 3243,
       6783, 5073, 5101, 3145, 3179, 6733, 7556, 5089, 3205, 5047, 6299,
       6166, 3243, 6943, 5077, 5790, 3183, 3307, 5118, 5081, 6871, 4685,
       5640, 7700, 6236, 5810, 4649, 5098])

In [58]:
rsupport

array([0.08353333, 0.08361333, 0.07501333, 0.08209333, 0.09264   ,
       0.08224   , 0.04237333, 0.10924   , 0.04252   , 0.08274667,
       0.04204   , 0.07712   , 0.06822667, 0.04236   , 0.08482667,
       0.07586667, 0.08294667, 0.07372   , 0.09316   , 0.0758    ,
       0.04337333, 0.04324   , 0.09044   , 0.06764   , 0.06801333,
       0.04193333, 0.04238667, 0.08977333, 0.10074667, 0.06785333,
       0.04273333, 0.06729333, 0.08398667, 0.08221333, 0.04324   ,
       0.09257333, 0.06769333, 0.0772    , 0.04244   , 0.04409333,
       0.06824   , 0.06774667, 0.09161333, 0.06246667, 0.0752    ,
       0.10266667, 0.08314667, 0.07746667, 0.06198667, 0.06797333])

In [14]:
def supports_vectorized(itemsets, matrix, minSup=0):
    supports_dict = {iset: 0 for iset in itemsets}
    for iset in itemsets:
        supports_dict[iset] = sum(reduce((lambda x,y: x&y), [matrix[item] for item in iset]))
    rm_list = []
    for iset in supports_dict.keys():
        if supports_dict[iset] < minSup:
            rm_list.append(iset)
    for iset in rm_list:
        supports_dict.pop(iset)
    return supports_dict
    #occurence_col = matrix[itemset[0]]
    #for i in range(1, len(itemset)):
    #    occurence_col = occurence_col & matrix[itemset[i]]
    #return sum(occurence_col)

In [15]:
def supports(itemsets, matrix, minSup=0):
    supports_dict = {iset: 0 for iset in itemsets}
    for idx in matrix.index:
        for iset in itemsets:
            row = matrix.loc[idx]
            if reduce((lambda x,y: x&y), [row[item] for item in iset]) == 1:
                supports_dict[iset] += 1
    rm_list = []
    for iset in supports_dict.keys():
        if supports_dict[iset] < minSup:
            rm_list.append(iset)
    for iset in rm_list:
        supports_dict.pop(iset)
    return supports_dict

In [16]:
def powerset_one_less(iterable):
    sets = []
    s = list(iterable)
    for i in range(len(iterable)):
        sets.append(frozenset(s[0:i]+s[i+1:]))
    return sets

In [17]:
def candidate_gen(set_list):
    C = []
    for f1 in set_list:
        for f2 in set_list:
            if len(f1.union(f2)) == len(f1) + 1:
                c = f1.union(f2)
                if c not in C:
                    flag = True
                    for subset in powerset_one_less(c):
                        if subset != f1 and subset != f2 and subset not in set_list:
                            flag = False
                    if flag:
                        C.append(c)
    
    return C

In [18]:
def apriori(matrix, total_itemset, minRelativeSup):
    all_supports_dict = {}
    skylines = {}
    minSup = minRelativeSup*matrix.shape[0]
    all_supports_dict[1] = supports_vectorized([frozenset([i]) for i in total_itemset], matrix, minSup)
    for singleton in all_supports_dict[1]:
        skylines[singleton] = 1
    k = 2
    while(k <= len(total_itemset) and len(all_supports_dict[k-1].values()) > 0):
        C_k = candidate_gen(all_supports_dict[k-1])
        all_supports_dict[k] = supports_vectorized(C_k, matrix, minSup)
        for ilist in all_supports_dict[k]:
            skylines[ilist] = 1
            for subset in powerset_one_less(ilist):
                skylines[subset] = 0
        k += 1
    skyline_list = []
    for itemset in skylines.keys():
        if skylines[itemset] == 1:
            skyline_list.append(itemset)
    return all_supports_dict, skyline_list

In [19]:
supports_dict,skyline_list = apriori(bakery, list(range(50)), .01)
print(skyline_list)

[frozenset({6}), frozenset({8}), frozenset({10}), frozenset({13}), frozenset({20}), frozenset({21}), frozenset({25}), frozenset({26}), frozenset({30}), frozenset({34}), frozenset({38}), frozenset({39}), frozenset({1, 19}), frozenset({9, 4}), frozenset({5, 22}), frozenset({44, 14}), frozenset({27, 28}), frozenset({33, 42}), frozenset({0, 2, 46}), frozenset({35, 18, 3}), frozenset({49, 15, 7}), frozenset({16, 32, 45}), frozenset({17, 29, 47}), frozenset({37, 7, 11, 45}), frozenset({48, 36, 12, 31}), frozenset({40, 41, 43, 23, 24})]


In [20]:
def association_rules(supports_dict, skyline_list, minConfidence = 0):
    confidence_dict = {}
    for iset in skyline_list:
        if (len(iset) >= 2):
            for item in iset:
                right_set = frozenset([item])
                left_set = iset.difference(right_set)
                total_support = supports_dict[len(iset)][iset]
                left_support = supports_dict[len(left_set)][left_set]
                confidence = total_support/left_support
                if confidence > minConfidence:
                    confidence_dict[(left_set, item)] = confidence
    return confidence_dict

In [22]:
association_rules_dict = association_rules(supports_dict, skyline_list, .5)

In [23]:
def goods_lookup(item_id, goods):
    row = goods.loc[item_id]
    return row["Flavor"][1:-1] + " " + row["Food"][1:-1]
goods_lookup(1, goods)

'Lemon Cake'

In [25]:
association_rules_formatted_list = []
for entry in association_rules_dict.keys():
    left_side = [goods_lookup(i, goods) for i in entry[0]]
    right_side = goods_lookup(entry[1], goods)
    confidence = association_rules_dict[entry]
    association_rules_formatted_list.append((left_side, right_side, confidence))
association_rules_formatted_list

[(['Napoleon Cake'], 'Strawberry Cake', 0.5214308733483726),
 (['Truffle Cake'], 'Gongolais Cookie', 0.5340466926070039),
 (['Bottled Water'], 'Berry Tart', 0.5026595744680851),
 (['Tuile Cookie'], 'Marzipan Cookie', 0.5054261514028586),
 (['Marzipan Cookie'], 'Tuile Cookie', 0.5672062973414526),
 (['Cheese Croissant'], 'Orange Juice', 0.5238404151800194),
 (['Casino Cake', 'Chocolate Coffee'], 'Chocolate Cake', 0.9474082482027999),
 (['Chocolate Cake', 'Chocolate Coffee'], 'Casino Cake', 0.7580986981531941),
 (['Chocolate Cake', 'Casino Cake'], 'Chocolate Coffee', 0.9395872420262664),
 (['Cherry Tart', 'Opera Cake'], 'Apricot Danish', 0.9477405471872118),
 (['Opera Cake', 'Apricot Danish'], 'Cherry Tart', 0.9553765106910443),
 (['Cherry Tart', 'Apricot Danish'], 'Opera Cake', 0.7742340532395781),
 (['Coffee Eclair', 'Blackberry Tart'], 'Single Espresso', 0.74697912852435),
 (['Single Espresso', 'Coffee Eclair'], 'Blackberry Tart', 0.9222423146473779),
 (['Single Espresso', 'Blackberry

In [41]:
def mine_bakery_rules(filename, minRelativeSup, minConf, goods_filename):
    goods = pd.read_csv(goods_filename)
    bakery = pd.read_csv(filename, names=goods.index)
    supports_dict,skyline_list = apriori(bakery, list(goods.index), minRelativeSup)
    association_rules_dict = association_rules(supports_dict, skyline_list, .5)
    skylines_formatted = []
    for entry in skyline_list:
        skyline_set = [goods_lookup(i, goods) for i in entry]
        support = supports_dict[len(entry)][entry]
        skylines_formatted.append((skyline_set, support))
    arules_formatted = []
    for entry in association_rules_dict.keys():
        left_side = [goods_lookup(i, goods) for i in entry[0]]
        right_side = goods_lookup(entry[1], goods)
        confidence = association_rules_dict[entry]
        arules_formatted.append((left_side, right_side, confidence))
    return skylines_formatted, arules_formatted

In [75]:
skylines_final_5000, rules_final_5000 = mine_bakery_rules("5000-out2.csv", .01, .5, "goods.csv")
print("Skylines:" + "\n")
for item in skylines_final_5000:
    print(item)
print()
print("Association Rules:" + "\n")
for item in rules_final_5000:
    print(item)

Skylines:

(['Chocolate Eclair'], 191)
(['Vanilla Eclair'], 230)
(['Almond Tart'], 193)
(['Apricot Tart'], 211)
(['Pecan Tart'], 222)
(['Ganache Cookie'], 194)
(['Chocolate Meringue'], 226)
(['Vanilla Meringue'], 199)
(['Almond Croissant'], 228)
(['Chocolate Croissant'], 216)
(['Almond Bear Claw'], 214)
(['Blueberry Danish'], 200)
(['Lemon Cake', 'Lemon Tart'], 168)
(['Napoleon Cake', 'Strawberry Cake'], 211)
(['Truffle Cake', 'Gongolais Cookie'], 236)
(['Bottled Water', 'Berry Tart'], 183)
(['Marzipan Cookie', 'Tuile Cookie'], 248)
(['Cheese Croissant', 'Orange Juice'], 215)
(['Chocolate Cake', 'Casino Cake', 'Chocolate Coffee'], 156)
(['Apricot Danish', 'Cherry Tart', 'Opera Cake'], 204)
(['Single Espresso', 'Blackberry Tart', 'Coffee Eclair'], 143)
(['Blueberry Tart', 'Apricot Croissant', 'Hot Coffee'], 164)
(['Chocolate Tart', 'Walnut Cookie', 'Vanilla Frappuccino'], 133)
(['Almond Twist', 'Coffee Eclair', 'Apple Pie', 'Hot Coffee'], 154)
(['Cherry Soda', 'Apple Danish', 'Apple Tar

In [74]:
skylines_final_20000, rules_final_20000 = mine_bakery_rules("20000-out2.csv", .01, .5, "goods.csv")
print("Skylines:" + "\n")
for item in skylines_final_20000:
    print(item)
print()
print("Association Rules:" + "\n")
for item in rules_final_20000:
    print(item)

Skylines:

(['Chocolate Eclair'], 852)
(['Vanilla Eclair'], 854)
(['Almond Tart'], 811)
(['Apricot Tart'], 855)
(['Pecan Tart'], 831)
(['Ganache Cookie'], 866)
(['Chocolate Meringue'], 890)
(['Vanilla Meringue'], 848)
(['Almond Croissant'], 841)
(['Chocolate Croissant'], 892)
(['Almond Bear Claw'], 885)
(['Blueberry Danish'], 823)
(['Lemon Cake', 'Lemon Tart'], 740)
(['Napoleon Cake', 'Strawberry Cake'], 891)
(['Truffle Cake', 'Gongolais Cookie'], 867)
(['Bottled Water', 'Berry Tart'], 714)
(['Marzipan Cookie', 'Tuile Cookie'], 971)
(['Cheese Croissant', 'Orange Juice'], 878)
(['Chocolate Cake', 'Casino Cake', 'Chocolate Coffee'], 678)
(['Apricot Danish', 'Cherry Tart', 'Opera Cake'], 820)
(['Single Espresso', 'Blackberry Tart', 'Coffee Eclair'], 539)
(['Blueberry Tart', 'Apricot Croissant', 'Hot Coffee'], 652)
(['Chocolate Tart', 'Walnut Cookie', 'Vanilla Frappuccino'], 565)
(['Almond Twist', 'Coffee Eclair', 'Apple Pie', 'Hot Coffee'], 562)
(['Cherry Soda', 'Apple Danish', 'Apple Tar

In [73]:
skylines_final_75000, rules_final_75000 = mine_bakery_rules("75000-out2.csv", .01, .5, "goods.csv")
print("Skylines:" + "\n")
for item in skylines_final_75000:
    print(item)
print()
print("Association Rules:" + "\n")
for item in rules_final_75000:
    print(item)

Skylines:

(['Chocolate Eclair'], 3178)
(['Vanilla Eclair'], 3189)
(['Almond Tart'], 3153)
(['Apricot Tart'], 3177)
(['Pecan Tart'], 3253)
(['Ganache Cookie'], 3243)
(['Chocolate Meringue'], 3145)
(['Vanilla Meringue'], 3179)
(['Almond Croissant'], 3205)
(['Chocolate Croissant'], 3243)
(['Almond Bear Claw'], 3183)
(['Blueberry Danish'], 3307)
(['Lemon Cake', 'Lemon Tart'], 2764)
(['Napoleon Cake', 'Strawberry Cake'], 3236)
(['Truffle Cake', 'Gongolais Cookie'], 3294)
(['Bottled Water', 'Berry Tart'], 2835)
(['Marzipan Cookie', 'Tuile Cookie'], 3819)
(['Cheese Croissant', 'Orange Juice'], 3230)
(['Chocolate Cake', 'Casino Cake', 'Chocolate Coffee'], 2504)
(['Apricot Danish', 'Cherry Tart', 'Opera Cake'], 3083)
(['Single Espresso', 'Blackberry Tart', 'Coffee Eclair'], 2040)
(['Blueberry Tart', 'Apricot Croissant', 'Hot Coffee'], 2462)
(['Chocolate Tart', 'Walnut Cookie', 'Vanilla Frappuccino'], 2007)
(['Almond Twist', 'Coffee Eclair', 'Apple Pie', 'Hot Coffee'], 2094)
(['Cherry Soda', 'A